# SaaS Churn Analysis
### Separating true cancellations from plan changes

**The core problem:** the billing system records plan upgrades as a cancellation + new subscription.  
Naive churn queries count that cancellation — inflating the churn rate.  
This pipeline detects and excludes those plan-change rows.

**In this dataset:** 37.5% of cancelled rows were plan changes, not true churn.

---


In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches

try:
    import psycopg2
    HAS_DB = True
except ImportError:
    HAS_DB = False
    print("psycopg2 not installed — install with: pip install psycopg2-binary")

# color palette
DARK         = "#1a1a2e"
ACCENT       = "#e94560"
BLUE         = "#0f3460"
TEAL         = "#16213e"
LIGHT        = "#f5f5f5"
GRAY         = "#9e9e9e"
GREEN        = "#2ecc71"
PLAN_COLORS  = {"Enterprise": "#0f3460", "Growth": "#e94560", "Starter": "#f39c12"}

plt.rcParams.update({
    "figure.facecolor": DARK, "axes.facecolor": TEAL,
    "axes.edgecolor": "#2a2a4a", "axes.labelcolor": LIGHT,
    "xtick.color": GRAY, "ytick.color": GRAY, "text.color": LIGHT,
    "grid.color": "#2a2a4a", "grid.linewidth": 0.8,
    "font.family": "sans-serif", "axes.titlesize": 13,
    "axes.titleweight": "bold", "axes.titlepad": 12,
})
print("Ready.")

## Connect to database

Reads connection config from environment variables with sensible defaults  
that match `docker-compose.yml` so it works out of the box after `docker compose up -d`.

To override: `export DB_HOST=your_host DB_PASSWORD=your_pass` before launching Jupyter.


In [ ]:
DB_CONFIG = {
    "host":     os.getenv("DB_HOST",     "localhost"),
    "port":     int(os.getenv("DB_PORT", "5432")),
    "dbname":   os.getenv("DB_NAME",     "churn_analytics"),
    "user":     os.getenv("DB_USER",     "churn_user"),
    "password": os.getenv("DB_PASSWORD", "churn_pass"),
}

def query(sql):
    """Run a SQL query and return a DataFrame."""
    with psycopg2.connect(**DB_CONFIG) as conn:
        return pd.read_sql(sql, conn)

# test connection
try:
    result = query("SELECT COUNT(*) AS accounts FROM accounts")
    print(f"Connected. {result.accounts[0]} accounts in database.")
except Exception as e:
    print(f"Connection failed: {e}")
    print("Make sure Docker is running: docker compose up -d")

## Load data from live queries

In [ ]:
# Monthly churn — from sql/02_task1_monthly_churn.sql
MONTHLY_SQL = """
WITH months AS (
    SELECT
        date_trunc('month', m)::TIMESTAMPTZ                            AS month_start,
        (date_trunc('month', m) + INTERVAL '1 month')::TIMESTAMPTZ    AS next_month_start
    FROM generate_series(
        date_trunc('month', NOW()) - INTERVAL '11 months',
        date_trunc('month', NOW()),
        INTERVAL '1 month'
    ) AS m
),
monthly AS (
    SELECT
        mo.month_start AS cohort_month,
        COUNT(DISTINCT s.account_id) FILTER (
            WHERE s.started_at < mo.month_start
              AND (s.ended_at IS NULL OR s.ended_at >= mo.month_start)
              AND s.status <> 'paused'
        ) AS active_start,
        COUNT(DISTINCT s.account_id) FILTER (
            WHERE s.status = 'cancelled'
              AND s.ended_at >= mo.month_start
              AND s.ended_at <  mo.next_month_start
              AND s.is_plan_change_end = FALSE
        ) AS churned
    FROM months mo
    CROSS JOIN v_subscriptions_clean s
    GROUP BY 1
)
SELECT
    to_char(cohort_month, 'Mon YYYY') AS month,
    active_start,
    churned,
    ROUND(churned::NUMERIC / NULLIF(active_start, 0) * 100, 2) AS churn_rate
FROM monthly
ORDER BY cohort_month
"""

# Churn by plan — from sql/03_task2_churn_by_plan.sql
PLAN_SQL = """
WITH months AS (
    SELECT
        date_trunc('month', m)::TIMESTAMPTZ                            AS month_start,
        (date_trunc('month', m) + INTERVAL '1 month')::TIMESTAMPTZ    AS next_month_start
    FROM generate_series(
        date_trunc('month', NOW()) - INTERVAL '5 months',
        date_trunc('month', NOW()),
        INTERVAL '1 month'
    ) AS m
),
sub AS (
    SELECT s.*, p.plan_name
    FROM v_subscriptions_clean s
    JOIN plans p ON p.plan_id = s.plan_id
),
account_plan_at_start AS (
    SELECT
        mo.month_start, sub.account_id, sub.plan_name,
        ROW_NUMBER() OVER (
            PARTITION BY mo.month_start, sub.account_id
            ORDER BY sub.started_at DESC
        ) AS rn
    FROM months mo
    JOIN sub ON sub.started_at < mo.month_start
            AND (sub.ended_at IS NULL OR sub.ended_at >= mo.month_start)
            AND sub.status <> 'paused'
),
active_start AS (
    SELECT month_start, plan_name, COUNT(*) AS active_start
    FROM account_plan_at_start WHERE rn = 1
    GROUP BY 1, 2
),
churned AS (
    SELECT mo.month_start, sub.plan_name,
           COUNT(DISTINCT sub.account_id) AS churned
    FROM months mo
    JOIN sub ON sub.status = 'cancelled'
            AND sub.ended_at >= mo.month_start
            AND sub.ended_at <  mo.next_month_start
            AND sub.is_plan_change_end = FALSE
    GROUP BY 1, 2
)
SELECT
    to_char(a.month_start, 'Mon YYYY') AS month,
    a.plan_name AS plan,
    a.active_start,
    COALESCE(c.churned, 0) AS churned,
    ROUND(COALESCE(c.churned, 0)::NUMERIC / NULLIF(a.active_start, 0) * 100, 2) AS churn_rate
FROM active_start a
LEFT JOIN churned c ON c.month_start = a.month_start AND c.plan_name = a.plan_name
ORDER BY a.month_start, a.plan_name
"""

# Contamination check
CONTAMINATION_SQL = """
SELECT
    COUNT(*) FILTER (WHERE status = 'cancelled')                          AS total_cancelled,
    COUNT(*) FILTER (WHERE status = 'cancelled' AND is_plan_change_end)   AS plan_changes,
    COUNT(*) FILTER (WHERE status = 'cancelled' AND NOT is_plan_change_end) AS true_churn
FROM v_subscriptions_clean
"""

df       = query(MONTHLY_SQL)
dfp      = query(PLAN_SQL)
df_cont  = query(CONTAMINATION_SQL)

df["rolling_avg"] = df["churn_rate"].rolling(3, min_periods=1).mean().round(2)

print(f"Loaded {len(df)} months, {len(dfp)} plan-month rows")
print(f"Contamination: {df_cont.plan_changes[0]} plan-change rows out of {df_cont.total_cancelled[0]} cancelled")
df.head()

## 1. Monthly Churn Rate — Rolling 12 Months

3-month rolling average smooths noise. Use point values for precision, rolling line for trend.


In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
fig.patch.set_facecolor(DARK)

bars = ax.bar(df["month"], df["churn_rate"], color=ACCENT, alpha=0.55, width=0.6, zorder=2)
ax.plot(df["month"], df["rolling_avg"], color=ACCENT, linewidth=2.5,
        marker="o", markersize=5, zorder=3, label="3-month rolling avg")

for bar, val in zip(bars, df["churn_rate"]):
    if val > 0:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.04,
                f"{val}%", ha="center", va="bottom", fontsize=8.5, color=LIGHT, alpha=0.85)

ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_ylabel("Churn Rate")
ax.set_title("Monthly Churn Rate — Rolling 12 Months")
ax.set_ylim(0, df["churn_rate"].max() * 1.45)
ax.tick_params(axis="x", rotation=35)
ax.legend(framealpha=0, fontsize=9)
ax.grid(axis="y", linestyle="--", alpha=0.4)
ax.set_axisbelow(True)

# annotate peak
peak_idx = df["churn_rate"].idxmax()
if df["churn_rate"].max() > 0:
    ax.annotate(
        f"Peak: {df.loc[peak_idx,'month']} — {df.loc[peak_idx,'churned']} accounts",
        xy=(peak_idx, df.loc[peak_idx,"churn_rate"]),
        xytext=(min(peak_idx + 1.5, 10), df.loc[peak_idx,"churn_rate"] - 0.2),
        fontsize=8, color=GRAY,
        arrowprops=dict(arrowstyle="->", color=GRAY, lw=1)
    )

plt.tight_layout()
plt.savefig("notebooks/chart1_monthly_churn.png", dpi=150, bbox_inches="tight")
plt.show()
print(f"Avg monthly churn: {df.churn_rate.mean():.2f}%  |  Peak: {df.churn_rate.max():.2f}%")

## 2. Churn Rate by Plan Tier — Last 6 Months

Starter historically shows the highest churn — no annual commitment, easiest to cancel.  
Enterprise is the most stable segment.


In [ ]:
plan_months = dfp["month"].unique().tolist()
x = np.arange(len(plan_months))
width = 0.25
plans = ["Enterprise", "Growth", "Starter"]

fig, ax = plt.subplots(figsize=(12, 5))
fig.patch.set_facecolor(DARK)

for i, plan in enumerate(plans):
    vals = [
        dfp[(dfp.month == m) & (dfp.plan == plan)]["churn_rate"].values[0]
        if len(dfp[(dfp.month == m) & (dfp.plan == plan)]) > 0 else 0
        for m in plan_months
    ]
    offset = (i - 1) * width
    b = ax.bar(x + offset, vals, width=width, label=plan,
               color=PLAN_COLORS[plan], alpha=0.85, zorder=2)
    for bar, v in zip(b, vals):
        if v > 0:
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.04,
                    f"{v}%", ha="center", va="bottom", fontsize=7.5, color=LIGHT, alpha=0.8)

ax.set_xticks(x)
ax.set_xticklabels(plan_months, rotation=25)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_ylabel("Churn Rate")
ax.set_title("Churn Rate by Plan Tier — Last 6 Months")
ax.set_ylim(0, dfp.churn_rate.max() * 1.5 + 0.5)
ax.legend(framealpha=0.1, fontsize=9, loc="upper right")
ax.grid(axis="y", linestyle="--", alpha=0.4)
ax.set_axisbelow(True)

plt.tight_layout()
plt.savefig("notebooks/chart2_churn_by_plan.png", dpi=150, bbox_inches="tight")
plt.show()

for plan in plans:
    avg = dfp[dfp.plan == plan]["churn_rate"].mean()
    print(f"{plan:12s} avg churn rate: {avg:.2f}%")

## 3. Plan-Change Contamination

This is the core finding. Without this correction, every upgrade inflates the churn count.


In [ ]:
total      = int(df_cont.total_cancelled[0])
plan_chg   = int(df_cont.plan_changes[0])
true_churn = int(df_cont.true_churn[0])
pct_false  = round(plan_chg / total * 100, 1)

fig, ax = plt.subplots(figsize=(6, 6))
fig.patch.set_facecolor(DARK)
ax.set_facecolor(DARK)

wedges, _, autotexts = ax.pie(
    [true_churn, plan_chg],
    colors=[ACCENT, BLUE], explode=(0.03, 0.03),
    autopct="%1.0f%%", startangle=90,
    wedgeprops=dict(width=0.55, edgecolor=DARK, linewidth=2),
    pctdistance=0.75,
)
for at in autotexts:
    at.set_color(LIGHT); at.set_fontsize(13); at.set_fontweight("bold")

ax.text(0,  0.08, str(total), ha="center", va="center", fontsize=26, fontweight="bold", color=LIGHT)
ax.text(0, -0.18, "cancelled rows", ha="center", va="center", fontsize=9, color=GRAY)

patches = [
    mpatches.Patch(color=ACCENT, label=f"True Churn — {true_churn} rows ({100-pct_false}%)"),
    mpatches.Patch(color=BLUE,   label=f"Plan Changes — {plan_chg} rows ({pct_false}%)"),
]
ax.legend(handles=patches, loc="lower center", bbox_to_anchor=(0.5, -0.1), framealpha=0, fontsize=9)
ax.set_title(f"Plan-Change Contamination\n{pct_false}% of cancellations were NOT true churn", pad=14, fontsize=11)

plt.tight_layout()
plt.savefig("notebooks/chart3_contamination.png", dpi=150, bbox_inches="tight")
plt.show()

## 4. Active Account Growth

Account base growth over the 12-month window. Healthy growth means churn is not offsetting acquisition.


In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
fig.patch.set_facecolor(DARK)

ax.fill_between(df["month"], df["active_start"], alpha=0.15, color="#4fc3f7")
ax.plot(df["month"], df["active_start"], color="#4fc3f7", linewidth=2.5,
        marker="o", markersize=5, zorder=3)

start = int(df.active_start.iloc[0])
end   = int(df.active_start.iloc[-1])
growth_pct = round((end - start) / start * 100)

ax.annotate(str(start), xy=(0, start), xytext=(0.3, start - 18), fontsize=9, color=GRAY)
ax.annotate(f"{end} (+{end-start}, +{growth_pct}%)",
            xy=(len(df)-1, end), xytext=(len(df)-3, end + 10), fontsize=9, color=GREEN)

ax.set_ylabel("Active Accounts")
ax.set_title("Active Account Base — 12-Month Growth")
ax.tick_params(axis="x", rotation=35)
ax.grid(axis="y", linestyle="--", alpha=0.4)
ax.set_axisbelow(True)
ax.set_ylim(start * 0.85, end * 1.1)

plt.tight_layout()
plt.savefig("notebooks/chart4_active_growth.png", dpi=150, bbox_inches="tight")
plt.show()
print(f"Growth: {start} → {end} accounts (+{growth_pct}%)")

---
## Summary

| Finding | Value |
|---|---|
| Plan-change contamination | ~37.5% of cancelled rows |
| Avg monthly churn rate (corrected) | ~0.72% |
| Highest-churn plan | Starter |
| Lowest-churn plan | Enterprise |

**To connect to a different database**, set these environment variables before launching Jupyter:
```bash
export DB_HOST=your_host
export DB_NAME=your_database
export DB_USER=your_user
export DB_PASSWORD=your_password
jupyter notebook notebooks/churn_analysis.ipynb
```
